# BD wrapper tutorial - plot distance PDFs

This notebook is intended to guide users through an example case of the BD wrapper to establish distances for a large number of position-position-velocity (PPV) sources.

For more information exceeding this tutorial we recommend taking a look at the following papers and resources:

- For a description about the Bayesian Distance Calculator see:
> - [Reid et al. 2016](https://ui.adsabs.harvard.edu/abs/2016ApJ...823...77R/abstract)
> - [Reid et al. 2019](https://ui.adsabs.harvard.edu/abs/2019ApJ...885..131R/abstract)

- For a description about new priors included in the BD wrapper and a test of its performance see: 
> - Riener et al. 2020b (coming to arXiv soon)

See also:
- ``Parameter_settings.ipynb`` notebook for an overview about and explanation of the main parameters of the BD wrapper
- ``Tutorial-batch_distance_estimation.ipynb`` notebook for a tutorial for batch distance estimations

In [ ]:
# TODO: import necessary libraries and plotting functions

import os

from astropy.table import Table

For this tutorial we will illustrate the distance calcutions for a single PPV source and show how to plot the distance probability density functions output by the BDC. We will specify the source coordinates and its kinematic properties in the `input_data.dat` file that is located in the `data` directory. This file is essentially a table with a single row, with the first row giving the column names that we need to specify for the BD wrapper.

Lets take a look at the file:

In [ ]:
t = Table.read(os.path.join('..', 'data', 'input_data.dat'), format='ascii')
t

This table includes seven columns giving the Galactic longitude and latitude position (**GLON**, **GLAT**), its radial velocity (**Vlsr**) and the associated error (**e_VLSR**), its associated velocity dispersion (**vel_disp**), solutions to the kinematic distance ambiguity (**KDA**), and the name of the source (**Name**).
Note that the only essential columns are **GLON**, **GLAT**, and **Vlsr**; the remaining columns are optional.

We now run the distance calculation with the BD wrapper for the source in `input_data.dat`. For this we run the `example_plot_distance_pdf.py` script contained in the `tutorials` directory. Running the following cell displays the content of this script.

In [ ]:
%pycat example_plot_distance_pdf.py

For this example we use the BDC v2.4 with settings used in Riener et al. (2020b). With `b.check_for_kda_solutions = True` we use the information from literature distance solutions to inform the prior for the kinematic distance ambiguity (KDA). By default all tables from the `KDA_info` directory are used to compute the prior for the KDA.

If we uncomment the lines

```python
b.colname_kda = 'KDA'
b.kda_weight = 0.75
```

we use the KDA information that is supplied by the `input_data.dat` file directly, weighting it by a factor of 0.75. For example if we put `KDA = 'F'`, we specify that the source is associated with the far kinematic distance solution and set $P_{\text{far}} = 0.875$. For `b.kda_weight = 1.0`, we would have gotten $P_{\text{far}} = 1$.


In addition to or instead of the KDA prior we can also use the size-linewidth prior to inform $P_{\text{far}}$ by setting

```python
b.prior_velocity_dispersion = True
```

In this case we also have to specify `b.colname_vel_disp` and `b.beam` and have to supply a velocity dispersion value associated with the source. See the `Parameter_settings.ipynb` notebook and Sect. 3.3 in Riener et al. (2020b) for more information.

Now we execute the script in the terminal, by going into the `tutorials` directory and running
```shell
python example_plot_distance_pdf.py
```

### Results

If the script was running successfully it produced a couple of files in the `results` directory. Lets first take a look at the first ten entries of the `input_data_distance_results.dat` table:

In [ ]:
t = Table.read(os.path.join('results', 'input_data_distance_results.dat'), format='ascii')
t

This table is an appended version of the original `input_data.dat` file, in which the additional columns contain the full information on the distance results. The additional columns are:
- **comp**: the number of reported possible distance results for the source. The BDC v2.4 always reports two distance results (`comp = 2`), so the full distance info for each source occupies two rows in the table.
- **dist**: the calculated heliocentric distance in [kpc]
- **e_dist**: the uncertainty in **dist** in [kpc]
- **prob**: the estimated probability for the distance result. 
- **arm**: the associated spiral arm or Galactic feature; `...` denotes no association.
- **c_u, c_v, c_w**: Coordinates centered at the sun. The **c_u** coordiante is along GLON=0°, the **c_v** coordinate is along GLON=90°, the **c_w** coordinate is along GLAT=90°. These three values are all given in [kpc].
- **p_far**: probability that the source is located at the far kinematic distance 
- **KDA_ref**: reference to literature work whose distance(s) helped to inform p_far. See Table A.1 in Riener et al. (2020b) for more information on all literature distance solutions we compare in this step.
- **kDist_1**: near kinematic distance solution in [kpc]
- **kDist_2**: far kinematic distance solution in [kpc]
- **rgal** Galactocentric distance in [kpc]

Next, lets take a look at the the `input_data_distance_results_p_max.dat` table:

In [ ]:
t = Table.read(os.path.join('results', 'input_data_distance_results_p_max.dat'), format='ascii')
t

This table shows only the final selected distance component for the source. Note the additional **flag** column, whose value indicates how the distance was chosen. There are five possible flags for the distance choice:
- (0): the distance assignment yielded only one distance solution
- (1): the associated Gaussian fit of one distance solution did not satisfy the criterion for the amplitude threshold
- (2): the distance solution with the highest probability (i.e. the highest integrated intensity of the associated Gaussian fit) was chosen
- (3): the distance solution with the lowest absolute distance error was chosen
- (4): the near kinematic distance solution was picked randomly

See Sect. 3.4 and Appendix C.3 in Riener et al. 2020b for more details on the choice of the distance solution.

### Additional files

In addition to tables of the distance results the `results` directory also contains a plot of the distance PDF and a few temporary files produced by the BDC, that can be valuable for debugging and have diagnostic value.

The following new files have been produced if `b.save_temporary_files = True`:
- `_bdc_results_list.pickle`: a pickled file containing the distance results from the BD wrapper
- `test_arm_latitude_pdf.dat`: contains the values of the distance PDF resulting from the combination of the SA + GL prior
- `test_arm_pdf.dat`: contains the values of the distance PDF for the SA prior
- `test_arm_ranges.dat`: specifies the associated Galactic features
- `test_final_distance_pdf.dat`: contains the values of the final distance PDF, which is a combination of all the individual distance PDFs
- `test_kinematic_distance_pdf.dat`: contains the values of the distance PDF for the KD prior
- `test_latitude_pdf.dat`: contains the values of the distance PDF for the GL prior
- `test_parallaxes_pdf.dat`: contains the values of the distance PDF for the PS prior
- `test_pm_bee_distance_pdf.dat`: contains the values of the distance PDF for the PM (Galactic latitude) prior
- `test_pm_ell_distance_pdf.dat`: contains the values of the distance PDF for the PM (Galactic longitude) prior
- `test_sources_info.inp`: input data file for the BDC
- `test_summary.prt`: summary of the BDC results
- `test.f`: BDC Fortran file for the source
- `test.out`: compiled BDC Fortran file for the source
- `test.prt`: extended summary of the BDC results

if `b.plot_probability = True` we produce the `test.pdf` file, which is a plot of the distance PDF similar as in Figs. 2 and 3 in Riener et al. (2020b). This essentially plots the information contained in the `test_arm_latitude_pdf.dat`, `test_arm_ranges.dat`, `test_final_distance_pdf.dat`, `test_kinematic_distance_pdf.dat`, `test_latitude_pdf.dat`, `test_parallaxes_pdf.dat`, and `test_summary.prt` files. See Riener et al. (2020b) for more information about the plot.

### Next steps

Take a look at the `Parameter_settings.ipynb` notebook for more information about possible settings for the BD wrapper.
Try to change some of the settings in the `example_plot_distance_pdf.py` and check what consequences it has on the distance PDFs. For example, you can play around with the weights for the priors (`prob_sa`, `prob_ps`, `prob_gl`, `prob_kd`) to check how this influences the distance outcome. You can also add multiple rows to the `input_data.dat` file but beware that temporary files and plots will be produced for each individual source.

Also take a look at the other tutorial (`Tutorial-batch_distance_estimation.ipynb`) that shows how to obtain distances for a larger table of input sources.